# 22.目标导向的强化学习（goal-oriented reinforcement learning，GoRL）
> 之前学习的如 **PPO、SAC** 等经典的深度强化学习算法，智能体的行为通常是根据固定的奖励函数来学习的，只关注单一目标。算法训练完后，在使用它们时只能完成一个特定的任务。
> 而在许多实际任务场景中，目标是复杂的，涉及到 **多个子目标或必须在长期内达成的目标**。之前传统的强化学习算法往往不容易训练出有效的策略。
> 例如，在悬崖漫步环境中，若目标位置变成了右上角，便只能重新训练一个策略。同一个策略无法完成一系列不同的目标。

> **目标导向的强化学习（GoRL）** 的概念并非源自单一的原始论文，而是逐渐在多个领域中演化和扩展的一个 **思想**：
> - 让智能体在前往 **最终目标** 的训练过程中将所经历的 **失败经验** 也作为一种 **目标** 进行学习，使智能体理解 **错误目标** 是如何达成的，最终学习一种 **策略**：在应对 **不同的目标（goal）** 依然奏效。以此来解决较为复杂的 **复合任务**。

> 本节介绍该思想下的一种经典算法 [事后经验回放（hindsight experience replay，HER）](https://arxiv.org/abs/1707.01495)：主要是为了解决 **稀疏奖励**

## 22.1 GoRL 中的马尔可夫决策过程（MDP）
> 在 **目标导向的强化学习** 中，**MDP** 的定义使用一个扩充过的元组：
> $$\langle\mathcal{S},\mathcal{A},P,r_{g},\mathcal{G},\phi\rangle$$

- **$\mathcal{S}$**：状态空间（state space），环境中所有可能的状态
- **$\mathcal{A}$**：动作空间（action space），智能体在每个状态下可采取的动作集合
- **$P$**：状态转移概率分布 $P(s'|s,a)$，描述动作如何影响状态变化
- **$r_g$**：**目标条件奖励函数**，即奖励不仅依赖于状态和动作，还依赖于当前目标 $g \in \mathcal{G}$
- **$\mathcal{G}$**：目标空间（goal space），任务中所有可能的目标集合
- **$\phi$**：目标映射函数，将状态 $s \in \mathcal{S}$ 映射到$g \in \mathcal{G}$，用于判断某个状态是否达成目标
> 可见，这是一个 **带目标条件的 MDP**（Goal-conditioned MDP，GC-MDP）。相比于传统 RL 只关心“最大化奖励”，在 GoRL 中，**奖励函数和策略都依赖于目标**，比如目标是“到达某个位置”，那么奖励设计和学习策略都随目标改变。因此 **状态价值函数$V(s,g)$** 也是基于目标的，**动作状态价值函数$Q(s,a,g)$** 同理。

> 举个例子：**机器人导航**
- $\mathcal{S}$：房间里机器人的位置（状态空间）
- $\mathcal{A}$：机器人能走的动作（上下左右）
- $P$：走一步后到达新位置的概率
- $r_g$：如果机器人到达目标位置 $g$，奖励 = 1，否则 = 0
- $\mathcal{G}$：房间里所有可能的目标位置 $g_i$
- $\phi(s)$：提取当前位置，判断是否等于目标

> 如此，为了机器人能够能根据不同的目标 $g$ 导航到对应位置，**GoRL** 的目标为优化一个 **通用的策略 $\pi(a|s,g)$**，使以下目标函数最大化：
$$\mathbb{E}_{(s_0,g)\sim\nu_0}\left[V^\pi\left(s_0,g\right)\right]$$
- $\nu_0$ 为环境中初始状态 $s_0$ 与目标 $g$ 的联合分布

## 22.2 HER 算法
> **事后经验回放（hindsight experience replay，HER）** 算法于 2017 年 **神经信息处理系统（Neural Information Processing Systems，NeurIPS）** 大会中被提出，成为 GoRL 的一大经典方法。

> 其提出主要是为了解决 **稀疏奖励** 问题：智能体采样的 *绝大多数轨迹都拿不到正奖励，学习信号极少*
> **HER** 把一次 **未达成原目标** 的轨迹，事后改写目标为轨迹中实际达成的某个目标（hindsight goal），并重算奖励；这样“失败轨迹”变成了“成功样本”，极大提高正样本密度与信号利用率，这样理论上就可以从稀疏和二元的奖励中进行样本高效学习，避免了复杂的奖励工程的需要

> **HER** 可以与任意的 **off-policy RL** 算法（比如 DQN、DDPG 等）相结合

### HER 算法流程

1. **初始化**  
   - 初始化策略 $\pi_\theta$，回放池 $R$，目标空间 $\mathcal{G}$（可自定义），以及训练的迭代次数 $E$

2. **采样轨迹**  
   - 在每个 episode 中，从环境中采样初始状态 $s_0$ 和目标 $g$，然后进行与环境的交互
   - 在每一步 $t$，根据策略 $\pi_\theta$ 选择动作 $a_t$，并获得下一个状态 $s_{t+1}$ 和奖励 $r_t$
   - 将状态、动作、奖励、下一个状态和目标$(s_i, a_i, r_i, s'_i, g_i)$存储到经验回放池 $R$ 中

3. **回放和目标替换**  
   - 从回放池中采样一批 $N$ 个样本 $(s_i, a_i, r_i, s'_i, g_i)$
   - **选择一个状态 $s''_i$**，并用 $\phi(s''_i)$ 计算新目标 $g'_i$
   - 计算新的奖励 $r'_i = r_{g'_i}(s_i, a_i, s'_i)$
   - 将原始元组替换为新的元组 $(s_i, a_i, r'_i, s'_i, g'_i)$

4. **更新策略**  
   - 使用经过替换目标后的样本来更新策略 $\pi_\theta$

5. **循环迭代**  
   - 不断采样 → **替换目标** → 更新策略，直到收敛


#### 状态 $s''$ 代表了一个新的目标，通过目标映射函数 $\phi(s'')$，可以生成新的目标
> 在**选择状态 $s''$** 作为新的目标时，**HER** 提出了 **三种常用方案**：  
> 1. **future**：从同一轨迹中在时间上位于 $s$ 之后的状态 $s''$，最常用
> 2. **episode**：从同一轨迹中的任意状态 $s''$ 选择作为目标
> 3. **random**：从回放池中随机选择状态作为目标

在实践中，**future 策略**效果最佳，因为它确保替换目标是轨迹中确实达到的状态。  


#### 直观例子：机器人需要把一个物体放到 **指定位置 A**
- 在一次 episode 里，它最终没有把物体放到位置 A，而是放到了 **位置 B**
- 在 **传统 RL** 中，这条轨迹没有用（奖励全是 0）
- 在 **HER** 中：  
  - 我们把位置 B 当成新的目标 $g'$
  - 这时轨迹就是 “成功地把物体放到位置 B”
  - 于是智能体依然能从这次经验中学习




### HER 算法代码实践
> 策略优化使用 **DDPG** 算法

导入相关库：

In [1]:
# 基本库
import numpy as np
import random
from tqdm import tqdm
import collections

from utils.smoothing import moving_average
# 神经网络
import torch
import torch.nn.functional as F

#### 自定义二维平面环境
> 定义一个二维网格世界，维度的范围都是 $[0,5]$
> 初始，智能体都位于 $(0,0)$，环境将自动从矩形区域 $3.5<=x,y<=4.5$ 内生成一个目标。每个时刻智能体可以选择**纵向和横向**分别移动范围 $[-1,1]$ 作为这一时刻的动作
> 当智能体距离目标足够近的时候，它将收到 $0$ 的奖励并结束任务，否则奖励为 $1$ 
> 每一条序列的最大长度为 50

> ![二维网格世界环境示意图](Illustrations/二维网格世界环境示意图.png)


In [2]:
class WorldEnv:
    def __init__(self):
        self.count = None
        self.state = None
        self.goal = None
        self.distance_threshold = 0.15  # 如果智能体与目标的距离小于等于该阈值，就认为到达了目标
        self.action_bound = 1

    def reset(self):  # 重置环境
        # 生成一个目标状态, 坐标范围是[3.5～4.5, 3.5～4.5]
        self.goal = np.array(
            [4 + random.uniform(-0.5, 0.5), 4 + random.uniform(-0.5, 0.5)])
        self.state = np.array([0, 0])  # 初始状态
        self.count = 0
        return np.hstack((self.state, self.goal))

    def step(self, action):
        # 防止越界
        action = np.clip(action, -self.action_bound, self.action_bound)
        x = max(0, min(5, self.state[0] + action[0]))
        y = max(0, min(5, self.state[1] + action[1]))
        self.state = np.array([x, y])
        self.count += 1

        dis = np.sqrt(np.sum(np.square(self.state - self.goal)))  # 计算智能体当前位置与目标位置之间的欧几里得距离
        reward = -1.0 if dis > self.distance_threshold else 0
        if dis <= self.distance_threshold or self.count == 50:
            done = True
        else:
            done = False

        return np.hstack((self.state, self.goal)), reward, done

#### 使用 DDPG 算法中的网络结构

In [3]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, action_dim)
        self.action_bound = action_bound  # action_bound是环境可以接受的动作最大值

    def forward(self, x):
        x = F.relu(self.fc2(F.relu(self.fc1(x))))
        return torch.tanh(self.fc3(x)) * self.action_bound


class QValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1)  # 拼接状态和动作
        x = F.relu(self.fc2(F.relu(self.fc1(cat))))
        return self.fc3(x)

#### DDPG算法
> 相比在 **倒立摆环境** 运行的 **DDPG算法**，这里的环境中动作有 2 维，导致一行代码的不同

In [4]:
class DDPG:
    """ DDPG算法 """
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound, actor_lr, critic_lr, sigma, tau, gamma, device):
        
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.target_actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.target_critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        
        # 初始化目标网络并设置和主网络相同的参数
        self.target_critic.load_state_dict(self.critic.state_dict())
        self.target_actor.load_state_dict(self.actor.state_dict())
        
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)
        
        self.gamma = gamma
        self.sigma = sigma  # 高斯噪声的标准差设为0
        self.tau = tau  # 目标网络软更新参数
        self.action_dim = action_dim
        self.device = device

    def take_action(self, state):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        #------------------------------------代码区别--------------------------------------------
        action = self.actor(state).detach().cpu().numpy()[0]
        #------------------------------------代码区别--------------------------------------------
        # 给动作添加噪声，增加探索
        action = action + self.sigma * np.random.randn(self.action_dim)
        return action
            

    # 软更新
    def soft_update(self, net, target_net):
        for param_target, param in zip(target_net.parameters(), net.parameters()):
            param_target.data.copy_(param_target.data * (1.0 - self.tau) + param.data * self.tau)
            

    def update(self, transition_dict):
        
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)
        #------------------------------------代码区别--------------------------------------------
        actions = torch.tensor(transition_dict['actions'], dtype=torch.float).to(self.device)
        #------------------------------------代码区别--------------------------------------------
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device)

        
        next_q_values = self.target_critic(next_states, self.target_actor(next_states))
        q_targets = rewards + self.gamma * next_q_values * (1 - dones)
        
        critic_loss = torch.mean(F.mse_loss(self.critic(states, actions), q_targets))
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        actor_loss = -torch.mean(self.critic(states, self.actor(states)))
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.soft_update(self.actor, self.target_actor)  # 软更新策略网络
        self.soft_update(self.critic, self.target_critic)  # 软更新价值网络

#### 专属经验回放池
> 不再存储每一步的数据，而是存储一整条轨迹

In [5]:
class Trajectory:
    """ 用来记录一条完整轨迹 """
    def __init__(self, init_state):
        self.states = [init_state]
        self.actions = []
        self.rewards = []
        self.dones = []
        self.length = 0

    def store_step(self, action, state, reward, done):
        self.actions.append(action)
        self.states.append(state)
        self.rewards.append(reward)
        self.dones.append(done)
        self.length += 1

# 保存多个轨迹，以供训练使用
class ReplayBufferTrajectory:
    """ 存储轨迹的经验回放池 """
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)  # 创建一个循环队列，最大长度为 capacity

    def add_trajectory(self, trajectory):
        self.buffer.append(trajectory)

    def size(self):
        return len(self.buffer)
    
    # 随机抽取一批轨迹
    def sample(self, batch_size, use_her, dis_threshold=0.15, her_ratio=0.8):
        batch = dict(states=[],
                     actions=[],
                     next_states=[],
                     rewards=[],
                     dones=[])
        for _ in range(batch_size):
            traj = random.sample(self.buffer, 1)[0]  # 一条轨迹 traj
            step_state = np.random.randint(traj.length)  # 一个时间步 step_state
            
            state = traj.states[step_state]
            next_state = traj.states[step_state + 1]
            action = traj.actions[step_state]
            reward = traj.rewards[step_state]
            done = traj.dones[step_state]

            if use_her and np.random.uniform() <= her_ratio:  # 启用 HER 并以一定的概率（her_ratio）应用 HER
                step_goal = np.random.randint(step_state + 1, traj.length + 1)
                goal = traj.states[step_goal][:2]  # 使用HER算法的future方案设置目标
                dis = np.sqrt(np.sum(np.square(next_state[:2] - goal)))
                reward = -1.0 if dis > dis_threshold else 0
                done = False if dis > dis_threshold else True
                state = np.hstack((state[:2], goal))
                next_state = np.hstack((next_state[:2], goal))

            batch['states'].append(state)
            batch['next_states'].append(next_state)
            batch['actions'].append(action)
            batch['rewards'].append(reward)
            batch['dones'].append(done)
            
        # 转换为 NumPy 数组
        batch['states'] = np.array(batch['states'])
        batch['next_states'] = np.array(batch['next_states'])
        batch['actions'] = np.array(batch['actions'])
        return batch

#### 采用 HER 的离线策略优化

***环境初始化：***

In [6]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
env = WorldEnv()

***超参数设置：***

In [7]:
state_dim = 4
action_dim = 2
action_bound = 1
hidden_dim = 128

actor_lr = 1e-3
critic_lr = 1e-3
sigma = 0.1
tau = 0.005
gamma = 0.98
device = torch.device("cuda") if torch.cuda.is_available() else torch.device( "cpu")

buffer_size = 10000
replay_buffer = ReplayBufferTrajectory(buffer_size)


agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, 
             actor_lr, critic_lr, sigma, tau, gamma, device)

batch_size = 256
minimal_episodes = 200  # 回收池中储存的一条轨迹对应一次迭代
n_train = 20
num_episodes = 2000

***测试与训练:***

In [8]:
return_list = []
for i in range(10):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            state = env.reset()
            traj = Trajectory(state)
            done = False
            while not done:
                action = agent.take_action(state)
                state, reward, done = env.step(action)
                episode_return += reward
                traj.store_step(action, state, reward, done)
            
            replay_buffer.add_trajectory(traj)
            return_list.append(episode_return)
            if replay_buffer.size() >= minimal_episodes:
                for _ in range(n_train):
                    transition_dict = replay_buffer.sample(batch_size, True)
                    agent.update(transition_dict)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / 10 * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-10:])
                })
            pbar.update(1)

Iteration 9: 100%|██████████| 200/200 [01:35<00:00,  2.10it/s, episode=2000, return=-3.700]


***采用 HER 的效果展示：***

In [9]:
import pandas as pd
episodes_list = list(range(len(return_list)))
mv_return = moving_average(return_list, 9)
# 创建 DataFrame
df2 = pd.DataFrame({'Episodes': episodes_list, 'Returns': mv_return})
# 保存为 CSV 文件
df2.to_csv('HER_two-dimensional-world-v0_mv_returns_data.csv', index=False)

In [1]:
import plotly.graph_objects as go
import pandas as pd
df = pd.read_csv('./Data/HER_two-dimensional-world-v0_mv_returns_data.csv')  # 从 CSV 文件中读取数据
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Episodes'], y=df['Returns'], mode='lines', name='Returns'))
fig.update_layout(
    title='HER on two-dimensional-world-v0',
    xaxis_title='Episodes',
    yaxis_title='Returns',
    showlegend=True
)
fig.show()

#### 对比实验：不采用 HER 重新构造数据，而是直接使用收集的数据训练一个策略（稀疏奖励环境）

In [11]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
env = WorldEnv()

replay_buffer = ReplayBufferTrajectory(buffer_size)
agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, actor_lr,
             critic_lr, sigma, tau, gamma, device)

In [12]:
return_list = []
for i in range(10):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            state = env.reset()
            traj = Trajectory(state)
            done = False
            while not done:
                action = agent.take_action(state)
                state, reward, done = env.step(action)
                episode_return += reward
                traj.store_step(action, state, reward, done)
            
            replay_buffer.add_trajectory(traj)
            return_list.append(episode_return)
            if replay_buffer.size() >= minimal_episodes:
                for _ in range(n_train):
                    # --------------------------不使用 HER 训练-------------------------------------
                    transition_dict = replay_buffer.sample(batch_size, False)
                    # --------------------------不使用 HER 训练-------------------------------------
                    agent.update(transition_dict)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / 10 * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-10:])
                })
            pbar.update(1)

Iteration 9: 100%|██████████| 200/200 [01:07<00:00,  2.98it/s, episode=2000, return=-45.300]


***不采用 HER 的效果展示（DDPG）：***

In [13]:
import pandas as pd
episodes_list = list(range(len(return_list)))
mv_return = moving_average(return_list, 9)
# 创建 DataFrame
df2 = pd.DataFrame({'Episodes': episodes_list, 'Returns': mv_return})
# 保存为 CSV 文件
df2.to_csv('DDPG_two-dimensional-world-v0_mv_returns_data.csv', index=False)

In [2]:
import plotly.graph_objects as go
import pandas as pd
df = pd.read_csv('./Data/DDPG_two-dimensional-world-v0_mv_returns_data.csv')  # 从 CSV 文件中读取数据
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Episodes'], y=df['Returns'], mode='lines', name='Returns'))
fig.update_layout(
    title='DDPG on two-dimensional-world-v0',
    xaxis_title='Episodes',
    yaxis_title='Returns',
    showlegend=True
)
fig.show()

***对比可见，使用 HER 算法后，效果有显著提升***

#### 总结

1. 从代码实践中可以知道 **HER** 只是一个经验回放的修改方式，并没有对策略网络和价值网络的架构做出任何修改（在后续的部分 GoRL 研究中，也有对网络进行重构的）。
2. **HER** 通过重新对历史轨迹设置其目标（使用 future 方案）而使得奖励信号更加稠密，进而从原本失败的数据中学习到使“新任务”成功的经验，提升训练的稳定性和样本效率。